# P2 - Weather Data(APIs)

Group 9
- DONGCHEN WU     - 214604524
- RUNZE XU        - 220682498
- SUSHMIT RICHARD - 220105599

In [3]:
import sqlalchemy as sa
from datetime import timedelta,datetime
import pandas as pd
import numpy as np
import requests
import json
import time
import os
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from shapely.geometry import Point, Polygon
import overpy
import re
from translate import Translator

In [14]:
#Database Credentials
my_db_secret = {
    'drivername': 'postgresql+psycopg2', 
    'host': 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port': '5432', 
    'username': 'sushmit9', 
    'password': '2023!Schulich',
    'database': 'sushmit9_db'
}

#Creating Databse URL
my_db_url = sa.engine.URL.create(
    drivername = my_db_secret['drivername'],
    username = my_db_secret['username'],
    password = my_db_secret['password'],
    host = my_db_secret['host'],
    port = my_db_secret['port'],
    database = my_db_secret['database'])

# Create the SQLAlchemy engine
my_db_engine = sa.create_engine(my_db_url)

# 1. Current Weather for Toronto

#### Fetching Data

In [3]:
# Data Source: https://openweathermap.org/current
lat= 43.6532
lon = -79.3832
api_key = "7bada83efe23d6a1b6204c7ee293a2f4"
api_url = 'https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&appid={}'.format(lat, lon, api_key)
data = requests.get(api_url)
data = data.json()
data    

{'coord': {'lon': -79.3839, 'lat': 43.6535},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 294.69,
  'feels_like': 294.91,
  'temp_min': 292.71,
  'temp_max': 296.04,
  'pressure': 1008,
  'humidity': 77},
 'visibility': 10000,
 'wind': {'speed': 6.17, 'deg': 350, 'gust': 9.26},
 'clouds': {'all': 75},
 'dt': 1691898778,
 'sys': {'type': 1,
  'id': 718,
  'country': 'CA',
  'sunrise': 1691835512,
  'sunset': 1691886411},
 'timezone': -14400,
 'id': 6167863,
 'name': 'Downtown Toronto',
 'cod': 200}

#### Feature Engineering

In [4]:
# Flatten the nested dictionaries
flattened_data = {
    'lon': data['coord']['lon'],
    'lat': data['coord']['lat'],
    'weather_id': data['weather'][0]['id'],
    'weather_main': data['weather'][0]['main'],
    'weather_description': data['weather'][0]['description'],
    'weather_icon': data['weather'][0]['icon'],
    'base': data['base'],
    'temp': data['main']['temp'],
    'feels_like': data['main']['feels_like'],
    'temp_min': data['main']['temp_min'],
    'temp_max': data['main']['temp_max'],
    'pressure': data['main']['pressure'],
    'humidity': data['main']['humidity'],
    'visibility': data['visibility'],
    'wind_speed': data['wind']['speed'],
    'wind_deg': data['wind']['deg'],
    'clouds_all': data['clouds']['all'],
    'dt': data['dt'],
    'sys_type': data['sys']['type'],
    'sys_id': data['sys']['id'],
    'sys_country': data['sys']['country'],
    'sys_sunrise': data['sys']['sunrise'],
    'sys_sunset': data['sys']['sunset'],
    'timezone': data['timezone'],
    'city_id': data['id'],
    'city_name': data['name'],
    'cod': data['cod']
    }

# Convert the flattened data to a DataFrame
df = pd.DataFrame(flattened_data, index=[0])
df

,lon,lat,weather_id,weather_main,weather_description,weather_icon,base,temp,feels_like,temp_min,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,city_id,city_name,cod
0,-79.3839,43.6535,803,Clouds,broken clouds,04n,stations,294.69,294.91,292.71,...,1691898778,1,718,CA,1691835512,1691886411,-14400,6167863,Downtown Toronto,200


In [5]:
# Convert UNIX timestamps to 'hh:mm:ss' format using lambda function
df['dt'] = df['dt'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
df['sys_sunrise'] = df['sys_sunrise'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
df['sys_sunset'] = df['sys_sunset'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
# Convert timezone shift to 'hhmm' format using lambda function
df['timezone'] = df['timezone'].apply(lambda shift: ('-' if shift < 0 else '') + '{:02d}{:02d}'.format(abs(shift) // 3600, (abs(shift) % 3600) // 60))

df

,lon,lat,weather_id,weather_main,weather_description,weather_icon,base,temp,feels_like,temp_min,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,city_id,city_name,cod
0,-79.3839,43.6535,803,Clouds,broken clouds,04n,stations,294.69,294.91,292.71,...,03:52:58,1,718,CA,10:18:32,00:26:51,-0400,6167863,Downtown Toronto,200


In [6]:
df.columns

Index(['lon', 'lat', 'weather_id', 'weather_main', 'weather_description',
       'weather_icon', 'base', 'temp', 'feels_like', 'temp_min', 'temp_max',
       'pressure', 'humidity', 'visibility', 'wind_speed', 'wind_deg',
       'clouds_all', 'dt', 'sys_type', 'sys_id', 'sys_country', 'sys_sunrise',
       'sys_sunset', 'timezone', 'city_id', 'city_name', 'cod'],
      dtype='object')

In [7]:
#Dropping unneceassry columns and intenal parameters
df.drop(['weather_id','weather_icon','base','sys_type','sys_id', 'cod'], axis=1, inplace=True)
df

,lon,lat,weather_main,weather_description,temp,feels_like,temp_min,temp_max,pressure,humidity,...,wind_speed,wind_deg,clouds_all,dt,sys_country,sys_sunrise,sys_sunset,timezone,city_id,city_name
0,-79.3839,43.6535,Clouds,broken clouds,294.69,294.91,292.71,296.04,1008,77,...,6.17,350,75,03:52:58,CA,10:18:32,00:26:51,-0400,6167863,Downtown Toronto


In [8]:
df.dtypes

lon                    float64
lat                    float64
weather_main            object
weather_description     object
temp                   float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure                 int64
humidity                 int64
visibility               int64
wind_speed             float64
wind_deg                 int64
clouds_all               int64
dt                      object
sys_country             object
sys_sunrise             object
sys_sunset              object
timezone                object
city_id                  int64
city_name               object
dtype: object

#### DATA INGESTION PIPELINE

In [10]:
# Function to fetch and ingest data - compiling everything above into a function
def fetch_and_ingest_data():
    lat= 43.6532
    lon = -79.3832
    api_key = "7bada83efe23d6a1b6204c7ee293a2f4"
    api_url = 'https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&appid={}'.format(lat, lon, api_key)
    data = requests.get(api_url)
    data = data.json()
    
    # Flatten the nested dictionaries
    flattened_data = {
    'lon': data['coord']['lon'],
    'lat': data['coord']['lat'],
    'weather_id': data['weather'][0]['id'],
    'weather_main': data['weather'][0]['main'],
    'weather_description': data['weather'][0]['description'],
    'weather_icon': data['weather'][0]['icon'],
    'base': data['base'],
    'temp': data['main']['temp'],
    'feels_like': data['main']['feels_like'],
    'temp_min': data['main']['temp_min'],
    'temp_max': data['main']['temp_max'],
    'pressure': data['main']['pressure'],
    'humidity': data['main']['humidity'],
    'visibility': data['visibility'],
    'wind_speed': data['wind']['speed'],
    'wind_deg': data['wind']['deg'],
    'clouds_all': data['clouds']['all'],
    'dt': data['dt'],
    'sys_type': data['sys']['type'],
    'sys_id': data['sys']['id'],
    'sys_country': data['sys']['country'],
    'sys_sunrise': data['sys']['sunrise'],
    'sys_sunset': data['sys']['sunset'],
    'timezone': data['timezone'],
    'city_id': data['id'],
    'city_name': data['name'],
    'cod': data['cod']
    }
    
    # Convert the flattened data to a DataFrame
    df = pd.DataFrame(flattened_data, index=[0])

    # Convert UNIX timestamps to 'hh:mm:ss' format using lambda function
    df['dt'] = df['dt'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
    df['sys_sunrise'] = df['sys_sunrise'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
    df['sys_sunset'] = df['sys_sunset'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
    
    # Convert timezone shift to 'hhmm' format using lambda function
    df['timezone'] = df['timezone'].apply(lambda shift: ('-' if shift < 0 else '') + '{:02d}{:02d}'.format(abs(shift) // 3600, (abs(shift) % 3600) // 60))

    
    #Dropping unneceassry columns and intenal parameters
    df.drop(['weather_id','weather_icon','base','sys_type','sys_id', 'cod'], axis=1, inplace=True)

    #Pushing dataframe to Database Schema
    df.to_sql(
    name      = 'Toronto_current_weather',
    con       = my_db_engine,
    schema    = 'Uploads',
    if_exists = 'append', # add new rows to the existing table
    index     = False,
    method    = 'multi',
    dtype     = {
        'lon'                 : sa.types.FLOAT,
        'lat'                 : sa.types.FLOAT,
        'weather_main'        : sa.types.VARCHAR(30),
        'weather_description' : sa.types.VARCHAR(30),
        'temp'                : sa.types.FLOAT,
        'feels_like'          : sa.types.FLOAT,
        'temp_min'            : sa.types.FLOAT,
        'temp_max'            : sa.types.FLOAT,
        'pressure'            : sa.types.INTEGER,
        'humidity'            : sa.types.INTEGER,
        'visibility'          : sa.types.INTEGER,
        'wind_speed'          : sa.types.FLOAT,
        'wind_deg'            : sa.types.INTEGER,
        'clouds_all'          : sa.types.INTEGER,
        'dt'                  : sa.types.VARCHAR(30),
        'sys_country'         : sa.types.VARCHAR(30),
        'sys_sunrise'         : sa.types.VARCHAR(30),
        'sys_sunset'          : sa.types.VARCHAR(30),
        'timezone'            : sa.types.VARCHAR(30),
        'city_id'             : sa.types.INTEGER,
        'city_name'           : sa.types.VARCHAR(30)
    }
    )

# Immediately run the data fetching and ingestion process
print("Running data fetching and ingestion process immediately...")
fetch_and_ingest_data()
print("Data fetching and ingestion process has been executed.")

Running data fetching and ingestion process immediately...
Data fetching and ingestion process has been executed.


In [12]:
# Schedule the data fetching and ingestion process to run daily from the next day onwards
# P.S. - This is meant to be executed in the background. 
def schedule_daily_task(hour, minute):
    while True:
        current_time = time.localtime()
        if current_time.tm_hour >= hour and current_time.tm_min >= minute:
            break
        time.sleep(60)  # Sleep for a minute

    while True:
        fetch_and_ingest_data()
        print("Data fetching and ingestion process has been scheduled and executed.")
        time.sleep(24 * 60 * 60)  # Sleep for a day

# Schedule the data fetching and ingestion process to run every day at 12:00 PM
schedule_daily_task(hour=12, minute=0)

# 2. Weather Forecasts for Toronto

#### Fetching Data

In [17]:
# Data Source: https://openweathermap.org/api/hourly-forecast
lat2= 43.6532
lon2 = -79.3832
api_key2 = "583b9776c32850d28350562414730cb1"
api_url2 = 'https://pro.openweathermap.org/data/2.5/forecast/hourly?lat={}&lon={}&appid={}'.format(lat2, lon2, api_key2)
data2 = requests.get(api_url2)
data2 = data2.json()
data2   

{'cod': '200',
 'message': 0,
 'cnt': 96,
 'list': [{'dt': 1691902800,
   'main': {'temp': 294.24,
    'feels_like': 294.36,
    'temp_min': 293.02,
    'temp_max': 294.24,
    'pressure': 1008,
    'sea_level': 1008,
    'grnd_level': 998,
    'humidity': 75,
    'temp_kf': 1.22},
   'weather': [{'id': 803,
     'main': 'Clouds',
     'description': 'broken clouds',
     'icon': '04n'}],
   'clouds': {'all': 60},
   'wind': {'speed': 3.19, 'deg': 314, 'gust': 8.76},
   'visibility': 10000,
   'pop': 0.02,
   'sys': {'pod': 'n'},
   'dt_txt': '2023-08-13 05:00:00'},
  {'dt': 1691906400,
   'main': {'temp': 293.73,
    'feels_like': 293.78,
    'temp_min': 292.51,
    'temp_max': 293.73,
    'pressure': 1008,
    'sea_level': 1008,
    'grnd_level': 999,
    'humidity': 74,
    'temp_kf': 1.22},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03n'}],
   'clouds': {'all': 45},
   'wind': {'speed': 2.75, 'deg': 296, 'gust': 8.04},
  

#### Feature Engineering

In [18]:
# Flatten the nested data
flat_data_list = []

for item in data2['list']:
    flat_data = {
        'dt': item['dt'],
        'temp': item['main']['temp'],
        'feels_like': item['main']['feels_like'],
        'temp_min': item['main']['temp_min'],
        'temp_max': item['main']['temp_max'],
        'pressure': item['main']['pressure'],
        'sea_level': item['main'].get('sea_level'),  # Check if key exists
        'grnd_level': item['main'].get('grnd_level'),  # Check if key exists
        'temp_kf': item['main'].get('temp_kf'),  # Check if key exists
        'humidity': item['main']['humidity'],
        'weather_id': item['weather'][0]['id'],
        'weather_main': item['weather'][0]['main'],
        'weather_description': item['weather'][0]['description'],
        'weather_icon': item['weather'][0]['icon'],
        'clouds_all': item['clouds']['all'],
        'wind_speed': item['wind']['speed'],
        'wind_deg': item['wind']['deg'],
        'gust': item['wind']['gust'],
        'visibility': item['visibility'],
        'pop': item['pop'],
        'sys_pod': item['sys']['pod'],
        'dt_txt': item['dt_txt'],
        'city_id': data2['city']['id'],
        'city_name': data2['city']['name'],
        'city_lat': data2['city']['coord']['lat'],
        'city_lon': data2['city']['coord']['lon'],
        'city_country': data2['city']['country'],
        'city_timezone': data2['city']['timezone'],
        'city_sunrise': data2['city']['sunrise'],
        'city_sunset': data2['city']['sunset']
    }
    flat_data_list.append(flat_data)

# Convert the flattened data to a DataFrame
df2 = pd.DataFrame(flat_data_list)

df2.head()

,dt,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,temp_kf,humidity,...,sys_pod,dt_txt,city_id,city_name,city_lat,city_lon,city_country,city_timezone,city_sunrise,city_sunset
0,1691902800,294.24,294.36,293.02,294.24,1008,1008,998,1.22,75,...,n,2023-08-13 05:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-14400,1691921979,1691972724
1,1691906400,293.73,293.78,292.51,293.73,1008,1008,999,1.22,74,...,n,2023-08-13 06:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-14400,1691921979,1691972724
2,1691910000,293.04,293.02,292.04,293.04,1009,1009,999,1.00,74,...,n,2023-08-13 07:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-14400,1691921979,1691972724
3,1691913600,292.17,292.06,291.57,292.17,1009,1009,999,0.60,74,...,n,2023-08-13 08:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-14400,1691921979,1691972724
4,1691917200,291.24,291.09,291.24,291.24,1010,1010,1000,0.00,76,...,n,2023-08-13 09:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-14400,1691921979,1691972724


In [19]:
df2.columns

Index(['dt', 'temp', 'feels_like', 'temp_min', 'temp_max', 'pressure',
       'sea_level', 'grnd_level', 'temp_kf', 'humidity', 'weather_id',
       'weather_main', 'weather_description', 'weather_icon', 'clouds_all',
       'wind_speed', 'wind_deg', 'gust', 'visibility', 'pop', 'sys_pod',
       'dt_txt', 'city_id', 'city_name', 'city_lat', 'city_lon',
       'city_country', 'city_timezone', 'city_sunrise', 'city_sunset'],
      dtype='object')

In [20]:
# Convert UNIX timestamps to 'hh:mm:ss' format using lambda function
df2['dt'] = df2['dt'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
df2['city_sunrise'] = df2['city_sunrise'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
df2['city_sunset'] = df2['city_sunset'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
# Convert timezone shift to 'hhmm' format using lambda function
df2['city_timezone'] = df2['city_timezone'].apply(lambda shift: ('-' if shift < 0 else '') + '{:02d}{:02d}'.format(abs(shift) // 3600, (abs(shift) % 3600) // 60))

df2.head(3)

,dt,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,temp_kf,humidity,...,sys_pod,dt_txt,city_id,city_name,city_lat,city_lon,city_country,city_timezone,city_sunrise,city_sunset
0,05:00:00,294.24,294.36,293.02,294.24,1008,1008,998,1.22,75,...,n,2023-08-13 05:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-0400,10:19:39,00:25:24
1,06:00:00,293.73,293.78,292.51,293.73,1008,1008,999,1.22,74,...,n,2023-08-13 06:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-0400,10:19:39,00:25:24
2,07:00:00,293.04,293.02,292.04,293.04,1009,1009,999,1.00,74,...,n,2023-08-13 07:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-0400,10:19:39,00:25:24


In [21]:
#Dropping unneceassry columns and intenal parameters
df2.drop(['temp_kf','weather_id','weather_icon','city_id'], axis=1, inplace=True)
df2.head(1)

,dt,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,weather_main,...,pop,sys_pod,dt_txt,city_name,city_lat,city_lon,city_country,city_timezone,city_sunrise,city_sunset
0,05:00:00,294.24,294.36,293.02,294.24,1008,1008,998,75,Clouds,...,0.02,n,2023-08-13 05:00:00,Downtown Toronto,43.6532,-79.3832,CA,-0400,10:19:39,00:25:24


In [22]:
df2.dtypes

dt                      object
temp                   float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure                 int64
sea_level                int64
grnd_level               int64
humidity                 int64
weather_main            object
weather_description     object
clouds_all               int64
wind_speed             float64
wind_deg                 int64
gust                   float64
visibility               int64
pop                    float64
sys_pod                 object
dt_txt                  object
city_name               object
city_lat               float64
city_lon               float64
city_country            object
city_timezone           object
city_sunrise            object
city_sunset             object
dtype: object

#### Data Ingestion Pipeline

In [24]:
# Function to fetch and ingest data - compiling everything above into a function
def fetch_and_ingest_data2():
    lat2= 43.6532
    lon2 = -79.3832
    api_key2 = "583b9776c32850d28350562414730cb1"
    api_url2 = 'https://pro.openweathermap.org/data/2.5/forecast/hourly?lat={}&lon={}&appid={}'.format(lat2, lon2, api_key2)
    data2 = requests.get(api_url2)
    data2 = data2.json()
   
    # Flatten the nested data
    flat_data_list = []
    
    for item in data2['list']:
        flat_data = {
        'dt': item['dt'],
        'temp': item['main']['temp'],
        'feels_like': item['main']['feels_like'],
        'temp_min': item['main']['temp_min'],
        'temp_max': item['main']['temp_max'],
        'pressure': item['main']['pressure'],
        'sea_level': item['main'].get('sea_level'),  # Check if key exists
        'grnd_level': item['main'].get('grnd_level'),  # Check if key exists
        'temp_kf': item['main'].get('temp_kf'),  # Check if key exists
        'humidity': item['main']['humidity'],
        'weather_id': item['weather'][0]['id'],
        'weather_main': item['weather'][0]['main'],
        'weather_description': item['weather'][0]['description'],
        'weather_icon': item['weather'][0]['icon'],
        'clouds_all': item['clouds']['all'],
        'wind_speed': item['wind']['speed'],
        'wind_deg': item['wind']['deg'],
        'gust': item['wind']['gust'],
        'visibility': item['visibility'],
        'pop': item['pop'],
        'sys_pod': item['sys']['pod'],
        'dt_txt': item['dt_txt'],
        'city_id': data2['city']['id'],
        'city_name': data2['city']['name'],
        'city_lat': data2['city']['coord']['lat'],
        'city_lon': data2['city']['coord']['lon'],
        'city_country': data2['city']['country'],
        'city_timezone': data2['city']['timezone'],
        'city_sunrise': data2['city']['sunrise'],
        'city_sunset': data2['city']['sunset']
        }
        flat_data_list.append(flat_data)
    
    # Convert the flattened data to a DataFrame
    df2 = pd.DataFrame(flat_data_list)

    # Convert UNIX timestamps to 'hh:mm:ss' format using lambda function
    df2['dt'] = df2['dt'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
    df2['city_sunrise'] = df2['city_sunrise'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
    df2['city_sunset'] = df2['city_sunset'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
    
    # Convert timezone shift to 'hhmm' format using lambda function
    df2['city_timezone'] = df2['city_timezone'].apply(lambda shift: ('-' if shift < 0 else '') + '{:02d}{:02d}'.format(abs(shift) // 3600, (abs(shift) % 3600) // 60))

    #Dropping unneceassry columns and intenal parameters
    df2.drop(['temp_kf','weather_id','weather_icon','city_id'], axis=1, inplace=True)

    #Pushing dataframe to Database Schema
    df2.to_sql(
    name      = 'Toronto_weather_forecasts',
    con       = my_db_engine,
    schema    = 'Uploads',
    if_exists = 'append', # add new rows to the existing table
    index     = False,
    method    = 'multi',
    dtype     = {
        'dt'                  : sa.types.VARCHAR(30),
        'temp'                : sa.types.FLOAT,
        'feels_like'          : sa.types.FLOAT,
        'temp_min'            : sa.types.FLOAT,
        'temp_max'            : sa.types.FLOAT,
        'pressure'            : sa.types.INTEGER,
        'sea_leve'            : sa.types.INTEGER,
        'grnd_level'          : sa.types.INTEGER,
        'humidity'            : sa.types.INTEGER,
        'weather_main'        : sa.types.VARCHAR(30),
        'weather_description' : sa.types.VARCHAR(30),
        'clouds_all'          : sa.types.INTEGER,
        'wind_speed'          : sa.types.FLOAT,
        'wind_deg'            : sa.types.INTEGER,
        'gust'                : sa.types.FLOAT,
        'visibility'          : sa.types.INTEGER,
        'pop'                 : sa.types.FLOAT,
        'sys_pod'             : sa.types.VARCHAR(30),
        'dt_txt'              : sa.types.VARCHAR(30),
        'city_name'           : sa.types.VARCHAR(30),
        'city_lon'            : sa.types.FLOAT,
        'city_lat'            : sa.types.FLOAT, 
        'city_country'        : sa.types.VARCHAR(30),
        'city_timezone'       : sa.types.VARCHAR(30),
        'city_sunrise'        : sa.types.VARCHAR(30),
        'city_sunset'         : sa.types.VARCHAR(30)        
        }
    )

# Immediately run the data fetching and ingestion process
print("Running data fetching and ingestion process immediately...")
fetch_and_ingest_data2()
print("Data fetching and ingestion process has been executed.")

Running data fetching and ingestion process immediately...
Data fetching and ingestion process has been executed.


In [149]:
# Schedule the data fetching and ingestion process to run daily from the next day onwards
# P.S. - This is meant to be executed in the background. 
def schedule_daily_task2(hour, minute):
    while True:
        current_time = time.localtime()
        if current_time.tm_hour >= hour and current_time.tm_min >= minute:
            break
        time.sleep(60)  # Sleep for a minute

    while True:
        fetch_and_ingest_data2()
        print("Data fetching and ingestion process has been scheduled and executed.")
        time.sleep(24 * 60 * 60)  # Sleep for a day

# Schedule the data fetching and ingestion process to run every day at 12:00 PM
schedule_daily_task2(hour=12, minute=0)

Data fetching and ingestion process has been scheduled and executed.


# 3. Weather Alerts

#### Fetching the Data

In [5]:
# Data Source for alerts: https://dd.weather.gc.ca/alerts/cap/
# Data Source for Canadian cities-post codes: https://raw.githubusercontent.com/offcial17/schulich_data_science/main/ca_geo_dimension.csv
url3 = "https://dd.weather.gc.ca/alerts/cap/"
city = pd.read_csv('https://raw.githubusercontent.com/offcial17/schulich_data_science/main/ca_geo_dimension.csv')

In [6]:
# This function is used to scrape webpages containing links to CAP files, identify the latest CAP file link based on the date in the link,
# and return the URL of the latest CAP file.

def find_latest_cap_url(base_url):
    # Send an HTTP GET request to the URL
    response = requests.get(base_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find all anchor (a) elements that point to CAP files
        cap_folder_links = soup.find_all("a", href=True, string=True)
        
        latest_cap_folder_link = None
        latest_date = None
        
        # Iterate through CAP folder links to find the latest one
        for link in cap_folder_links:
            href = link.get("href")
            if href.endswith("/") and len(href) == 9 and href[:-1].isdigit():
                date = href[:-1]
                if latest_date is None or date > latest_date:
                    latest_date = date
                    latest_cap_folder_link = href
        
        if latest_cap_folder_link:
            latest_cap_file_link = f"{base_url}{latest_cap_folder_link}"
            return latest_cap_file_link
        else:
            return None
    else:
        return None

In [7]:
# This function finds and returns the URL of the most recently modified folder on a webpage. 

# It constructs a new URL with sorting parameters ('?C=M;O=A') to ensure that the webpage's content is sorted by modification date. 

# C=M: This part of the query string specifies the "C" parameter with the value "M," which stands for "Sort by Last Modified." 
# This indicates that the contents of the directory should be sorted based on the last modified timestamp of the files.

# O=A: This part of the query string specifies the "O" parameter with the value "A," which stands for "Order in Ascending." 
# This indicates that the sorting order should be ascending, meaning that files will be listed in order from the oldest to the newest.

# This way, it can reliably assume that the last link corresponds to the most recently modified folder.

def last_modified(url):
    # find the last modified folder
    sorted_new_url = url + '?C=M;O=A'
    
    response = requests.get(sorted_new_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the links on the webpage
    links = soup.find_all('a')

    # Last link is latest folder
    return url + links[-1].get('href')

In [8]:
#  To automate the process of identifying, downloading and saving the latest weather alert in CAP format(XML file) from the URL.
l_cap_url = find_latest_cap_url(url3)

new = last_modified(l_cap_url)
new = last_modified(new)
new_cap = last_modified(new)


#Download and write into new file
response = requests.get(new_cap)

# Save the file to disk
with open('latest_file.cap', 'wb') as file:
    file.write(response.content)

new_cap

'https://dd.weather.gc.ca/alerts/cap/20230813/CWWG/04/T_WOCN11_C_CWWG_202308130419_1160968572.cap'

In [9]:
# To parse the XML file, navigate to specific elements within the XML structure using namespaces and get the alert description text
# Path to the XML file
cap_file_path ='latest_file.cap'

# Parse the XML file
tree = ET.parse(cap_file_path)

# Get the root element
root = tree.getroot()

# Print the root element's tag
print("Root element:", root.tag)

# English version
info = root.find('{urn:oasis:names:tc:emergency:cap:1.2}info') 
description  = info.find('{urn:oasis:names:tc:emergency:cap:1.2}description').text
print(description)

Root element: {urn:oasis:names:tc:emergency:cap:1.2}alert

Near zero visibility in fog is expected or occurring.

Dense fog patches have developed over portions of western Manitoba late this evening. Regions in this advisory that are currently experiencing no reductions in visibility should expect to see these fog patches develop during the overnight hours. 

The fog will lift through the morning hours on Sunday.

###

Visibility may be significantly and suddenly reduced to near zero.

Fog advisories are issued when near zero visibilities in fog are expected or occurring.

Please continue to monitor alerts and forecasts issued by Environment Canada. To report severe weather, send an email to MBstorm@ec.gc.ca or tweet reports using #MBStorm.



#### Feature Engineering

In [10]:
# To extracts and processes polygon coordinates from the XML file, converting them from a string format to a list of tuples 
# containing latitude and longitude values.

polygon_coords = info.find('{urn:oasis:names:tc:emergency:cap:1.2}area').find('{urn:oasis:names:tc:emergency:cap:1.2}polygon').text

# Convert the string to coordinates seperated by , and space
pairs = polygon_coords.split()
polygon_coords = [(float(x.split(',')[0]), float(x.split(',')[1])) for x in pairs]
print(polygon_coords)

[(51.6214, -101.2593), (51.5365, -101.2369), (51.4494, -101.2593), (51.4538, -101.1104), (51.4494, -100.9781), (51.4052, -100.9655), (51.3617, -100.9781), (51.3092, -100.9705), (51.258, -100.9765), (51.2573, -100.9824), (51.258, -100.9887), (51.2472, -100.9866), (51.2357, -100.9884), (51.2336, -101.0114), (51.2358, -101.0354), (51.2297, -101.0341), (51.2208, -101.0355), (51.2238, -101.0086), (51.2211, -100.976), (51.1785, -100.9506), (51.1254, -100.9765), (51.1105, -101.2887), (51.1255, -101.539), (51.2152, -101.5458), (51.2437, -101.5436), (51.2911, -101.5399), (51.3018, -101.5391), (51.3018, -101.5705), (51.4362, -101.5758), (51.5667, -101.5705), (51.5707, -101.4633), (51.5652, -101.4001), (51.6213, -101.4001), (51.6297, -101.334), (51.6214, -101.2593)]


In [11]:
# To filter cities from the provided City-Post Codes Dataset that are located within the polygon coordinates
# Create a polygon using the coordinates
polygon = Polygon(polygon_coords)

# Filter cities within the polygon
filtered_cities = []
for  _,row in city.iterrows():
    point = Point(row["latitude"], row["longitude"])
    if polygon.contains(point):
        filtered_cities.append(row["region"]) #the city name colunms

# Remove duplicate city names by converting the list to a set and back to a list
filtered_cities = list(set(filtered_cities))
print("Cities within the polygon:", filtered_cities)

Cities within the polygon: ['Roblin, Russell, Rossburn']


In [12]:
# To extract specific attributes from the XML data and store them in a DataFrame,
info_dict = {
    "language": info.find('{urn:oasis:names:tc:emergency:cap:1.2}language').text,
    "event": info.find('{urn:oasis:names:tc:emergency:cap:1.2}event').text,
    "responseType": info.find('{urn:oasis:names:tc:emergency:cap:1.2}responseType').text,
    "urgency": info.find('{urn:oasis:names:tc:emergency:cap:1.2}urgency').text,
    "severity": info.find('{urn:oasis:names:tc:emergency:cap:1.2}severity').text,
    "certainty": info.find('{urn:oasis:names:tc:emergency:cap:1.2}certainty').text,
    "audience": info.find('{urn:oasis:names:tc:emergency:cap:1.2}audience').text,
    "effective": info.find('{urn:oasis:names:tc:emergency:cap:1.2}effective').text,
    "expires": info.find('{urn:oasis:names:tc:emergency:cap:1.2}expires').text,
    "senderName": info.find('{urn:oasis:names:tc:emergency:cap:1.2}senderName').text,
    "headline": info.find('{urn:oasis:names:tc:emergency:cap:1.2}headline').text,
    "description": info.find('{urn:oasis:names:tc:emergency:cap:1.2}description').text.strip(),
    "cities": ", ".join(filtered_cities) # to include cities within the polu=ygon
}

# Convert the dictionary to a dataframe
df3 = pd.DataFrame([info_dict])

df3.head()

,language,event,responseType,urgency,severity,certainty,audience,effective,expires,senderName,headline,description,cities
0,en-CA,fog,Monitor,Future,Moderate,Possible,general public,2023-08-13T04:24:35-00:00,2023-08-13T16:59:39-00:00,Environment Canada,fog advisory in effect,Near zero visibility in fog is expected or occ...,"Roblin, Russell, Rossburn"


#### Data Ingestion Pipeline

In [15]:
# Function to fetch and ingest data - compiling everything above into a function
def fetch_and_ingest_data3():
    l_cap_url = find_latest_cap_url(url3)
    
    new = last_modified(l_cap_url)
    new = last_modified(new)
    new_cap = last_modified(new)

    #Download and write into new file
    response = requests.get(new_cap)
    
    # Save the file to disk
    with open('latest_file.cap', 'wb') as file:
        file.write(response.content)

    # Path to the XML file
    cap_file_path ='latest_file.cap'
    
    # Parse the XML file
    tree = ET.parse(cap_file_path)
    
    # Get the root element
    root = tree.getroot()
    
    # Get Alert Description
    info = root.find('{urn:oasis:names:tc:emergency:cap:1.2}info') 
    description  = info.find('{urn:oasis:names:tc:emergency:cap:1.2}description').text 
    
    polygon_coords = info.find('{urn:oasis:names:tc:emergency:cap:1.2}area').find('{urn:oasis:names:tc:emergency:cap:1.2}polygon').text
    
    # Convert the string to coordinates seperated by , and space
    pairs = polygon_coords.split()
    polygon_coords = [(float(x.split(',')[0]), float(x.split(',')[1])) for x in pairs]

    # Create a polygon using the coordinates
    polygon = Polygon(polygon_coords)
    
    # Filter cities within the polygon
    filtered_cities = []
    for  _,row in city.iterrows():
        point = Point(row["latitude"], row["longitude"])
    if polygon.contains(point):
        filtered_cities.append(row["region"]) #the city name colunms
    
    #Remove duplicate city names by converting the list to a set and back to a list
    filtered_cities = list(set(filtered_cities))

    # To extract specific attributes from the XML data and store them in a DataFrame,
    info_dict = {
    "language": info.find('{urn:oasis:names:tc:emergency:cap:1.2}language').text,
    "event": info.find('{urn:oasis:names:tc:emergency:cap:1.2}event').text,
    "responseType": info.find('{urn:oasis:names:tc:emergency:cap:1.2}responseType').text,
    "urgency": info.find('{urn:oasis:names:tc:emergency:cap:1.2}urgency').text,
    "severity": info.find('{urn:oasis:names:tc:emergency:cap:1.2}severity').text,
    "certainty": info.find('{urn:oasis:names:tc:emergency:cap:1.2}certainty').text,
    "audience": info.find('{urn:oasis:names:tc:emergency:cap:1.2}audience').text,
    "effective": info.find('{urn:oasis:names:tc:emergency:cap:1.2}effective').text,
    "expires": info.find('{urn:oasis:names:tc:emergency:cap:1.2}expires').text,
    "senderName": info.find('{urn:oasis:names:tc:emergency:cap:1.2}senderName').text,
    "headline": info.find('{urn:oasis:names:tc:emergency:cap:1.2}headline').text,
    "description": info.find('{urn:oasis:names:tc:emergency:cap:1.2}description').text.strip(),
    "cities": ", ".join(filtered_cities) # to include cities within the poluygon
    }
    
    # Convert the dictionary to a dataframe
    df3 = pd.DataFrame([info_dict])

    #Pushing dataframe to Database Schema
    df3.to_sql(
    name      = 'Weather_Alerts',
    con       = my_db_engine,
    schema    = 'Uploads',
    if_exists = 'append', # add new rows to the existing table
    index     = False,
    method    = 'multi',
    dtype     = {
        'language'    : sa.types.VARCHAR(10),
        'event'       : sa.types.VARCHAR(50),
        'responseType': sa.types.VARCHAR(30),
        'urgency'     : sa.types.VARCHAR(30),
        'severity'    : sa.types.VARCHAR(30),
        'certainty'   : sa.types.VARCHAR(30),
        'audience'    : sa.types.VARCHAR(50),
        'effective'   : sa.types.VARCHAR(50),
        'expires'     : sa.types.VARCHAR(50),
        'senderName'  : sa.types.VARCHAR(50),
        'headline'    : sa.types.VARCHAR(100),
        'description' : sa.types.VARCHAR(),
        'cities'      : sa.types.VARCHAR(50) 
        }
    )

# Immediately run the data fetching and ingestion process
print("Running data fetching and ingestion process immediately...")
fetch_and_ingest_data3()
print("Data fetching and ingestion process has been executed.")

Running data fetching and ingestion process immediately...
Data fetching and ingestion process has been executed.


In [16]:
# Schedule the data fetching and ingestion process to run daily from the next day onwards
# P.S. - This is meant to be executed in the background. 
def schedule_daily_task3(hour, minute):
    while True:
        current_time = time.localtime()
        if current_time.tm_hour >= hour and current_time.tm_min >= minute:
            break
        time.sleep(60)  # Sleep for a minute

    while True:
        fetch_and_ingest_data3()
        print("Data fetching and ingestion process has been scheduled and executed.")
        time.sleep(24 * 60 * 60)  # Sleep for a day

# Schedule the data fetching and ingestion process to run every day at 12:00 PM
schedule_daily_task3(hour=12, minute=0)